In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [3]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

In [7]:
SEQ_LEN_S = 200
WORD_NUM_S = 20000


In [8]:
SEQ_LEN_T = 200
WORD_NUM_T = 20000

# 2 model

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,Bidirectional,LSTM,Dense

In [15]:
##encoder part 
input_s = Input(shape=(SEQ_LEN_S,),name='input_src')
embed_s = Embedding(input_dim=WORD_NUM_S,output_dim=64)(input_s)
##bilstm
s = Bidirectional(LSTM(units=64,return_sequences=True))(embed_s)
encoder_out,encoder_h,encoder_c = LSTM(units=64,return_sequences=True,return_state=True)(s)

In [17]:
print(encoder_out)
print(encoder_h)
print(encoder_c)

Tensor("lstm_5/transpose_1:0", shape=(?, 200, 64), dtype=float32)
Tensor("lstm_5/while/Exit_2:0", shape=(?, 64), dtype=float32)
Tensor("lstm_5/while/Exit_3:0", shape=(?, 64), dtype=float32)


In [18]:
##decoder part
input_t = Input(shape=(SEQ_LEN_T,),name='input_tgt')
embed_t = Embedding(input_dim=WORD_NUM_T,output_dim=64)(input_t)
t = LSTM(units=64,return_sequences=True)(embed_t,initial_state=[encoder_h,encoder_c])
decoder_out = Dense(units=WORD_NUM_T,activation='softmax')(t)

In [19]:
nn = Model([input_s,input_t],decoder_out)

In [20]:
nn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_src (InputLayer)          [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 64)      1280000     input_src[0][0]                  
__________________________________________________________________________________________________
input_tgt (InputLayer)          [(None, 200)]        0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 200, 128)     66048       embedding_2[0][0]                
______________________________________________________________________________________________

In [21]:
nn.compile(optimizer=tf.keras.optimizers.Adam(),loss = tf.keras.losses.sparse_categorical_crossentropy,
          metrics=[tf.keras.metrics.sparse_categorical_accuracy,tf.keras.metrics.sparse_categorical_crossentropy])